In [27]:
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,recall_score,confusion_matrix,precision_score
from sklearn.svm import SVC

In [16]:
TRAIN =True
def extract_features(db):
    train_db=pd.read_csv("spectograms/MeanNormalized/compare18-heartbeat-0.32-0.16-128-"+db+".csv/train/compare18-heartbeat-0.32-0.16-128-"+db+".csv")
    val_db=pd.read_csv("spectograms/MeanNormalized/compare18-heartbeat-0.32-0.16-128-"+db+".csv/devel/compare18-heartbeat-0.32-0.16-128-"+db+".csv")
    train_Y=train_db.label_nominal
    val_Y=val_db.label_nominal
    train_X=train_db.iloc[:,4:]
    val_X=val_db.iloc[:,4:]
    
    
    timesteps=64
    latent_dim=8
    
    train_X=train_X.values.flatten().reshape(len(train_X),timesteps,372) #502,64,372
    input_dim=train_X.shape[2]
    
    if TRAIN:
        inputs = Input(shape=(timesteps, input_dim))
        encoded = LSTM(latent_dim)(inputs)

        decoded = RepeatVector(timesteps)(encoded)
        decoded = LSTM(input_dim, return_sequences=True)(decoded)

        sequence_autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded) #hidden representations 

        sequence_autoencoder.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
        sequence_autoencoder.fit(train_X,train_X,batch_size=32,epochs=150,callbacks=[early_stop])
        sequence_autoencoder.save("models/autoencoder_2layer_"+str(timesteps)+"_"+db+".model")

    else :
        sequence_autoender.load("models/autoencoder_2layer_"+str(timesteps)+"_"+db+".model")
    features_train_X=encoder.predict(train_X)
    features_val_X=encoder.predict(val_X.values.reshape(len(val_X),timesteps,372))
    return features_train_X,features_val_X,train_Y,val_Y

In [17]:
train_75_X,val_75_X,train_75_Y,val_75_Y=extract_features("75")
train_60_X,val_60_X,train_60_Y,val_60_Y=extract_features("60")
train_45_X,val_45_X,train_45_Y,val_45_Y=extract_features("45")
train_30_X,val_30_X,train_30_Y,val_30_Y=extract_features("30")
# all Y's are same

Epoch 1/150
502/502 [==============================] - 5s 10ms/step - loss: 0.1117
Epoch 2/150
502/502 [==============================] - 4s 8ms/step - loss: 0.0832
Epoch 3/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0805
Epoch 4/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0794
Epoch 5/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0789
Epoch 6/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0786
Epoch 7/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0780
Epoch 8/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0777
Epoch 9/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0775
Epoch 10/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0773
Epoch 11/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0770
Epoch 12/150
502/502 [==============================] - 4s 7ms/step - loss: 0.0770
Epoch 13/150

In [19]:

train_X=np.concatenate((train_30_X,train_45_X,train_60_X,train_75_X),axis=1)
val_X=np.concatenate((val_30_X,val_45_X,val_60_X,val_75_X),axis=1)

In [36]:
model = Sequential()
model.add(Dense(256, input_dim=8*4,activation="relu"))
model.add(Dense(256))
model.add(Dense(3))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
train_Y=to_categorical(train_75_Y)
val_Y=to_categorical(val_75_Y)

In [37]:
#early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit(train_X,train_Y,batch_size=2,epochs=50,validation_data=(val_X, val_Y))

Train on 502 samples, validate on 180 samples
Epoch 1/50
502/502 [==============================] - 3s 5ms/step - loss: 7.3568 - acc: 0.1952 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 6/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 7/50
502/502 [==============================] - 1s 2ms/step - loss: 7.2564 - acc: 0.2012 - val_loss: 7.3427 - val_acc: 0.2333
Epoch 8/50
502/502 [==========

KeyboardInterrupt: 

In [137]:
svm_model=SVC(C=1,kernel="rbf")
svm_model.fit(train_X,train_Y)
predictions=svm_model.predict(encoder.predict(val_X.values.reshape(len(val_X),64,372)))
print("Recall: ",recall_score(val_Y,predictions,average="weighted"))
print("Accuracy: ",accuracy_score(val_Y,predictions))
confusion_matrix(val_Y,predictions)

Epoch 1/30
502/502 [==============================] - 6s 11ms/step - loss: 0.1094
Epoch 2/30
502/502 [==============================] - 4s 7ms/step - loss: 0.0827
Epoch 3/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0805
Epoch 4/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0797
Epoch 5/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0791
Epoch 6/30
502/502 [==============================] - 4s 7ms/step - loss: 0.0785
Epoch 7/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0782
Epoch 8/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0778
Epoch 9/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0777
Epoch 10/30
502/502 [==============================] - 4s 7ms/step - loss: 0.0773
Epoch 11/30
502/502 [==============================] - 4s 8ms/step - loss: 0.0770
Epoch 12/30
502/502 [==============================] - 4s 7ms/step - loss: 0.0768
Epoch 13/30
502/502 [===

In [139]:
features_X=encoder.predict(train_X)

In [46]:
svm_model=SVC(C=1,kernel="rbf")
svm_model.fit(train_X,train_30_Y)
predictions=svm_model.predict(val_X)
print("Recall: ",recall_score(val_30_Y,predictions,average="weighted"))
print("Accuracy: ",accuracy_score(val_30_Y,predictions))
confusion_matrix(val_45_Y,predictions)

Recall:  0.5444444444444444
Accuracy:  0.5444444444444444


array([[ 0, 32,  0],
       [ 0, 98,  0],
       [ 0, 50,  0]], dtype=int64)